In [1]:
import os
import glob
import warnings

import dask
from dask.distributed import Client
from dask import delayed, compute

import numpy as np
import xarray as xr
import dfm_tools as dfmt

import zarr

/home/rortiz/.conda/envs/jupyter_dask/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dask.distributed import Client

client = Client("tcp://145.136.58.253:41437")
client

Connection method: Direct,
Dashboard: /proxy/8787/status,
Comm: tcp://145.136.58.253:41437,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: 4 minutes ago,Total memory: 0 B


In [3]:
client.wait_for_workers(n_workers=32)

In [4]:
input_regex = r'/projects/0/einf1300/saltis-wp3-1/C_Work/05_RMD_2019/computations/D02/D02AU_newobs/RMD_dflowfm_D02AU_*_map.nc'

In [5]:
from dask import delayed, compute

def check_file(file):
    file_name = file.split('/')[-1]
    try:
        test = xr.open_dataset(file)
        return None  # If the file is fine, return None
    except:
        warnings.warn(f'The file {file_name} seems to be corrupted. Check your inputs.', UserWarning)
        return file  # Return the file name if it's corrupted

def check_corrupted_files_parallel(input_regex):
    file_list = glob.glob(input_regex)

    # Create a list of delayed tasks
    tasks = [delayed(check_file)(file) for file in file_list]

    # Compute tasks in parallel
    results = compute(*tasks)

    # Filter out None values to get only corrupted files
    corrupted_files = [file for file in results if file is not None]

    return corrupted_files


In [6]:
%%time
corrupted_files = check_corrupted_files_parallel(input_regex)

CPU times: user 95 ms, sys: 25.6 ms, total: 121 ms
Wall time: 26.1 s


In [7]:
# I.x Check if there are any corrupted files in the listed files
if len(corrupted_files) > 0:
    print('There are corrupted files. These will be taken out of the file list for the next tests.')
    
# I.x Get the entire input list and remove the corrupted files
input_files = [f for f in glob.glob(input_regex) if not f in corrupted_files]

In [8]:
len(input_files)

96

In [9]:
%%time
ds = dfmt.open_partitioned_dataset(input_files, chunks={'time':1})

>> xu.open_dataset() with 96 partition(s): 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 : 310.37 sec
>> xu.merge_partitions() with 96 partition(s): 18.46 sec
>> some variables dropped with merging of partitions: ['mesh2d_face_x_bnd', 'mesh2d_face_y_bnd']
>> dfmt.open_partitioned_dataset() total: 328.84 sec
CPU times: user 29.5 s, sys: 7.06 s, total: 36.6 s
Wall time: 5min 28s


In [10]:
times = ds.time
print(len(times))
gridname = ds.ugrid.grid.name
print(gridname)

793
mesh2d


In [11]:
ds

<xarray.Dataset> Size: 451GB
Dimensions:                      (time: 793, mesh2d_nNodes: 133336,
                                  mesh2d_nFaces: 126438, mesh2d_nLayers: 36,
                                  Two: 2, mesh2d_nInterfaces: 37,
                                  mesh2d_nEdges: 259786)
Coordinates:
  * time                         (time) datetime64[ns] 6kB 2019-07-30T12:00:0...
    mesh2d_node_x                (mesh2d_nNodes) float64 1MB dask.array<chunksize=(133336,), meta=np.ndarray>
    mesh2d_node_y                (mesh2d_nNodes) float64 1MB dask.array<chunksize=(133336,), meta=np.ndarray>
    mesh2d_face_y                (mesh2d_nFaces) float64 1MB dask.array<chunksize=(126438,), meta=np.ndarray>
    mesh2d_flowelem_zw           (time, mesh2d_nFaces, mesh2d_nInterfaces) float32 15GB dask.array<chunksize=(1, 126438, 37), meta=np.ndarray>
    mesh2d_flowelem_zcc          (time, mesh2d_nFaces, mesh2d_nLayers) float32 14GB dask.array<chunksize=(1, 126438, 36), meta=np.ndarray>
    mesh2d_face_x                (mesh2d_nFaces) float64 1MB dask.array<chunksize=(126438,), meta=np.ndarray>
    mesh2d_edge_x                (mesh2d_nEdges) float64 2MB dask.array<chunksize=(259786,), meta=np.ndarray>
    mesh2d_edge_y                (mesh2d_nEdges) float64 2MB dask.array<chunksize=(259786,), meta=np.ndarray>
  * mesh2d_nEdges                (mesh2d_nEdges) int64 2MB 0 1 ... 259784 259785
  * mesh2d_nNodes                (mesh2d_nNodes) int64 1MB 0 1 ... 133334 133335
  * mesh2d_nFaces                (mesh2d_nFaces) int64 1MB 0 1 ... 126436 126437
Dimensions without coordinates: mesh2d_nLayers, Two, mesh2d_nInterfaces
Data variables: (12/42)
    projected_coordinate_system  int32 4B ...
    timestep                     (time) float32 3kB dask.array<chunksize=(1,), meta=np.ndarray>
    mesh2d_node_z                (mesh2d_nNodes) float64 1MB dask.array<chunksize=(133336,), meta=np.ndarray>
    mesh2d_waterdepth            (time, mesh2d_nFaces) float32 401MB dask.array<chunksize=(1, 126438), meta=np.ndarray>
    mesh2d_ucxa                  (time, mesh2d_nFaces) float32 401MB dask.array<chunksize=(1, 126438), meta=np.ndarray>
    mesh2d_bldepth               (mesh2d_nFaces) float64 1MB dask.array<chunksize=(126438,), meta=np.ndarray>
    ...                           ...
    mesh2d_edge_type             (mesh2d_nEdges) float64 2MB dask.array<chunksize=(259786,), meta=np.ndarray>
    mesh2d_flowlink_zu_bnd       (time, mesh2d_nEdges, mesh2d_nLayers, Two) float32 59GB dask.array<chunksize=(1, 259786, 36, 2), meta=np.ndarray>
    mesh2d_flowlink_zu           (time, mesh2d_nEdges, mesh2d_nLayers) float32 30GB dask.array<chunksize=(1, 259786, 36), meta=np.ndarray>
    mesh2d_windxu                (time, mesh2d_nEdges) float32 824MB dask.array<chunksize=(1, 259786), meta=np.ndarray>
    mesh2d_turkin1               (time, mesh2d_nEdges, mesh2d_nInterfaces) float32 30GB dask.array<chunksize=(1, 259786, 37), meta=np.ndarray>
    mesh2d_diu                   (time, mesh2d_nEdges, mesh2d_nLayers) float32 30GB dask.array<chunksize=(1, 259786, 36), meta=np.ndarray>

In [12]:
# > Select only the relevant variables in the new for gradient computation /
# > salinity variance analysis. {gridname}_node_z is necessary to keep the
# > nodal dimension.
ds = ds[[f'{gridname}_node_z', f'{gridname}_sa1', 
f'{gridname}_ucx', f'{gridname}_ucy', f'{gridname}_ucz', f'{gridname}_tem1',
f'{gridname}_windx', f'{gridname}_windy', 
f'{gridname}_vicwwu', f'{gridname}_dtcell']]

In [13]:
ds

<xarray.Dataset> Size: 147GB
Dimensions:              (mesh2d_nNodes: 133336, time: 793,
                          mesh2d_nFaces: 126438, mesh2d_nLayers: 36,
                          mesh2d_nEdges: 259786, mesh2d_nInterfaces: 37)
Coordinates:
  * time                 (time) datetime64[ns] 6kB 2019-07-30T12:00:00 ... 20...
    mesh2d_node_x        (mesh2d_nNodes) float64 1MB dask.array<chunksize=(133336,), meta=np.ndarray>
    mesh2d_node_y        (mesh2d_nNodes) float64 1MB dask.array<chunksize=(133336,), meta=np.ndarray>
    mesh2d_face_y        (mesh2d_nFaces) float64 1MB dask.array<chunksize=(126438,), meta=np.ndarray>
    mesh2d_flowelem_zw   (time, mesh2d_nFaces, mesh2d_nInterfaces) float32 15GB dask.array<chunksize=(1, 126438, 37), meta=np.ndarray>
    mesh2d_flowelem_zcc  (time, mesh2d_nFaces, mesh2d_nLayers) float32 14GB dask.array<chunksize=(1, 126438, 36), meta=np.ndarray>
    mesh2d_face_x        (mesh2d_nFaces) float64 1MB dask.array<chunksize=(126438,), meta=np.ndarray>
    mesh2d_edge_x        (mesh2d_nEdges) float64 2MB dask.array<chunksize=(259786,), meta=np.ndarray>
    mesh2d_edge_y        (mesh2d_nEdges) float64 2MB dask.array<chunksize=(259786,), meta=np.ndarray>
  * mesh2d_nEdges        (mesh2d_nEdges) int64 2MB 0 1 2 ... 259784 259785
  * mesh2d_nNodes        (mesh2d_nNodes) int64 1MB 0 1 2 ... 133334 133335
  * mesh2d_nFaces        (mesh2d_nFaces) int64 1MB 0 1 2 ... 126436 126437
Dimensions without coordinates: mesh2d_nLayers, mesh2d_nInterfaces
Data variables:
    mesh2d_node_z        (mesh2d_nNodes) float64 1MB dask.array<chunksize=(133336,), meta=np.ndarray>
    mesh2d_sa1           (time, mesh2d_nFaces, mesh2d_nLayers) float32 14GB dask.array<chunksize=(1, 126438, 36), meta=np.ndarray>
    mesh2d_ucx           (time, mesh2d_nFaces, mesh2d_nLayers) float32 14GB dask.array<chunksize=(1, 126438, 36), meta=np.ndarray>
    mesh2d_ucy           (time, mesh2d_nFaces, mesh2d_nLayers) float32 14GB dask.array<chunksize=(1, 126438, 36), meta=np.ndarray>
    mesh2d_ucz           (time, mesh2d_nFaces, mesh2d_nLayers) float32 14GB dask.array<chunksize=(1, 126438, 36), meta=np.ndarray>
    mesh2d_tem1          (time, mesh2d_nFaces, mesh2d_nLayers) float32 14GB dask.array<chunksize=(1, 126438, 36), meta=np.ndarray>
    mesh2d_windx         (time, mesh2d_nFaces) float32 401MB dask.array<chunksize=(1, 126438), meta=np.ndarray>
    mesh2d_windy         (time, mesh2d_nFaces) float32 401MB dask.array<chunksize=(1, 126438), meta=np.ndarray>
    mesh2d_vicwwu        (time, mesh2d_nEdges, mesh2d_nInterfaces) float32 30GB dask.array<chunksize=(1, 259786, 37), meta=np.ndarray>
    mesh2d_dtcell        (time, mesh2d_nFaces, mesh2d_nLayers) float32 14GB dask.array<chunksize=(1, 126438, 36), meta=np.ndarray>

In [14]:
print(ds.data_vars)
print()
for var_name, data_var in ds.data_vars.items():
    print(f"Variable: {var_name}")
    print(f"Dimensions: {data_var.dims}")
    print()

Data variables:
    mesh2d_node_z  (mesh2d_nNodes) float64 1MB dask.array<chunksize=(133336,), meta=np.ndarray>
    mesh2d_sa1     (time, mesh2d_nFaces, mesh2d_nLayers) float32 14GB dask.array<chunksize=(1, 126438, 36), meta=np.ndarray>
    mesh2d_ucx     (time, mesh2d_nFaces, mesh2d_nLayers) float32 14GB dask.array<chunksize=(1, 126438, 36), meta=np.ndarray>
    mesh2d_ucy     (time, mesh2d_nFaces, mesh2d_nLayers) float32 14GB dask.array<chunksize=(1, 126438, 36), meta=np.ndarray>
    mesh2d_ucz     (time, mesh2d_nFaces, mesh2d_nLayers) float32 14GB dask.array<chunksize=(1, 126438, 36), meta=np.ndarray>
    mesh2d_tem1    (time, mesh2d_nFaces, mesh2d_nLayers) float32 14GB dask.array<chunksize=(1, 126438, 36), meta=np.ndarray>
    mesh2d_windx   (time, mesh2d_nFaces) float32 401MB dask.array<chunksize=(1, 126438), meta=np.ndarray>
    mesh2d_windy   (time, mesh2d_nFaces) float32 401MB dask.array<chunksize=(1, 126438), meta=np.ndarray>
    mesh2d_vicwwu  (time, mesh2d_nEdges, mesh2d_nIn

In [15]:
# Create a dictionary to store the dimensions of each data variable
ATTRS_ARRAY_DIMENSIONS = {}

# Iterate over each data variable in the dataset and store the dimensions
for var_name, data_var in ds.data_vars.items():
    ATTRS_ARRAY_DIMENSIONS[var_name] = list(data_var.dims)

# Print the resulting dictionary
print(ATTRS_ARRAY_DIMENSIONS)

{'mesh2d_node_z': ['mesh2d_nNodes'], 'mesh2d_sa1': ['time', 'mesh2d_nFaces', 'mesh2d_nLayers'], 'mesh2d_ucx': ['time', 'mesh2d_nFaces', 'mesh2d_nLayers'], 'mesh2d_ucy': ['time', 'mesh2d_nFaces', 'mesh2d_nLayers'], 'mesh2d_ucz': ['time', 'mesh2d_nFaces', 'mesh2d_nLayers'], 'mesh2d_tem1': ['time', 'mesh2d_nFaces', 'mesh2d_nLayers'], 'mesh2d_windx': ['time', 'mesh2d_nFaces'], 'mesh2d_windy': ['time', 'mesh2d_nFaces'], 'mesh2d_vicwwu': ['time', 'mesh2d_nEdges', 'mesh2d_nInterfaces'], 'mesh2d_dtcell': ['time', 'mesh2d_nFaces', 'mesh2d_nLayers']}


In [45]:
import zarr
import os

# Expand the user path and initialize the Zarr store
out_file = os.path.expanduser('~/smalloutfile.zarr')

# Initialize the Zarr group before writing
store = zarr.DirectoryStore(out_file)
root = zarr.group(store, overwrite=True)
print(f"Zarr store initialized at: {out_file}")

# Select the 'mesh2d_sa1' variable from the dataset
mesh2d_sa1 = ds['mesh2d_sa1']

# Define the shape and chunks for the array using mesh2d_sa1
shape = mesh2d_sa1.shape  # Get the shape of the variable
chunks = (1,) + shape[1:]  # Chunk by 'time', with 1 for the 'time' dimension

# Print the shape and chunk size for verification
print(f"Shape: {shape}")
print(f"Chunks: {chunks}")

# Initialize the Zarr store and array with specific chunks
zarr_array = zarr.open(store, mode='w', shape=shape, chunks=chunks, dtype='float32')
print(f"Zarr store initialized with custom chunks at: {out_file}")

# Store the data into the Zarr array
zarr_array[:] = mesh2d_sa1.values  # Write the actual data to the Zarr array

print("Data has been written to the Zarr store.")


Zarr store initialized at: /home/rortiz/smalloutfile.zarr
Shape: (793, 126438, 36)
Chunks: (1, 126438, 36)
Zarr store initialized with custom chunks at: /home/rortiz/smalloutfile.zarr
Data has been written to the Zarr store.


In [24]:
def write_var2zarr(var_name):
    # Expand the user path and initialize the Zarr store
    out_file = f"~/{var_name}.zarr"
    out_file = os.path.expanduser(out_file)

    # Initialize the Zarr group before writing
    store = zarr.DirectoryStore(out_file)
    root = zarr.group(store, overwrite=True)
    print(f"Zarr store initialized at: {out_file}")

    # Select the variable (e.g. 'mesh2d_sa1') from the dataset
    var = ds[var_name]

    # Define the shape and chunks for the array using mesh2d_sa1
    shape = var.shape  # Get the shape of the variable
    chunks = (1,) + shape[1:]  # Chunk by 'time', with 1 for the 'time' dimension

    # Print the shape and chunk size for verification
    print(f"Shape: {shape}")
    print(f"Chunks: {chunks}")

    # Initialize the Zarr store and array with specific chunks
    zarr_array = zarr.open(store, mode='w', shape=shape, chunks=chunks, dtype='float32')
    print(f"Zarr store initialized with custom chunks at: {out_file}")

    # Store the data into the Zarr array
    zarr_array[:] = var.values  # Write the actual data to the Zarr array
    
    # Store the variable name and dimension names in the Zarr metadata
    zarr_array.attrs['variable_name'] = var_name
    zarr_array.attrs['dimension_names'] = list(var.dims)  # Storing dimension names

    # Store any additional metadata from the variable
    #zarr_array.attrs['units'] = var.attrs.get('units', 'unknown')  # Example of storing units if available

    print("Data has been written to the Zarr store.")

In [25]:
%%time
write_var2zarr(var_name)

Zarr store initialized at: /home/rortiz/mesh2d_sa1.zarr
Shape: (793, 126438, 36)
Chunks: (1, 126438, 36)
Zarr store initialized with custom chunks at: /home/rortiz/mesh2d_sa1.zarr


2024-09-06 13:08:38,009 - distributed.client - WARNING - Couldn't gather 303 keys, rescheduling (('rechunk-merge-857f4b0576337795909def9c35bf7c3e', 696, 0, 0), ('rechunk-merge-857f4b0576337795909def9c35bf7c3e', 546, 0, 0), ('rechunk-merge-857f4b0576337795909def9c35bf7c3e', 89, 0, 0), ('rechunk-merge-857f4b0576337795909def9c35bf7c3e', 282, 0, 0), ('rechunk-merge-857f4b0576337795909def9c35bf7c3e', 679, 0, 0), ('rechunk-merge-857f4b0576337795909def9c35bf7c3e', 651, 0, 0), ('rechunk-merge-857f4b0576337795909def9c35bf7c3e', 503, 0, 0), ('rechunk-merge-857f4b0576337795909def9c35bf7c3e', 752, 0, 0), ('rechunk-merge-857f4b0576337795909def9c35bf7c3e', 600, 0, 0), ('rechunk-merge-857f4b0576337795909def9c35bf7c3e', 9, 0, 0), ('rechunk-merge-857f4b0576337795909def9c35bf7c3e', 707, 0, 0), ('rechunk-merge-857f4b0576337795909def9c35bf7c3e', 50, 0, 0), ('rechunk-merge-857f4b0576337795909def9c35bf7c3e', 98, 0, 0), ('rechunk-merge-857f4b0576337795909def9c35bf7c3e', 347, 0, 0), ('rechunk-merge-857f4b0576

Data has been written to the Zarr store.
CPU times: user 29.7 s, sys: 5.69 s, total: 35.4 s
Wall time: 4min 50s


In [26]:
in_file = os.path.expanduser('~/mesh2d_sa1.zarr')
print(in_file)

/home/rortiz/mesh2d_sa1.zarr


In [27]:
import zarr

# Open the Zarr store as an array (not a group)
zstore = zarr.DirectoryStore(in_file)
zarr_array = zarr.open(zstore, mode='r')

# Convert the Zarr array back into an xarray.DataArray
data_xr = xr.DataArray(zarr_array)

# Print the dataset to ensure it's loaded correctly
#print(data_xr)
data_xr

<xarray.DataArray (dim_0: 793, dim_1: 126438, dim_2: 36)> Size: 14GB
array([[[       nan,        nan,        nan, ..., 0.07259721,
         0.07260624, 0.07262179],
        [       nan,        nan,        nan, ..., 0.13580248,
         0.13581675, 0.13585113],
        [       nan,        nan,        nan, ..., 0.07255246,
         0.07256211, 0.07257775],
        ...,
        [       nan,        nan,        nan, ..., 0.54407346,
         0.54407346, 0.5440735 ],
        [       nan,        nan,        nan, ..., 0.08566034,
         0.08569561, 0.08575928],
        [       nan,        nan,        nan, ..., 0.08564866,
         0.08566928, 0.08570115]],

       [[       nan,        nan,        nan, ..., 0.07265066,
         0.07265936, 0.07267458],
        [       nan,        nan,        nan, ..., 0.1358823 ,
         0.13589603, 0.13592994],
        [       nan,        nan,        nan, ..., 0.07260521,
         0.07261463, 0.07263006],
...
        [       nan,        nan,        nan, ..., 0.03569586,
         0.03569588, 0.03569592],
        [       nan,        nan,        nan, ..., 0.08613854,
         0.08614878, 0.08617063],
        [       nan,        nan,        nan, ..., 0.08607471,
         0.08607985, 0.08608955]],

       [[       nan,        nan,        nan, ..., 0.06734708,
         0.06735098, 0.06735997],
        [       nan,        nan,        nan, ..., 0.13130829,
         0.13131426, 0.13132927],
        [       nan,        nan,        nan, ..., 0.06727014,
         0.0672757 , 0.06728686],
        ...,
        [       nan,        nan,        nan, ..., 0.03935511,
         0.03935514, 0.03935517],
        [       nan,        nan,        nan, ..., 0.08614923,
         0.08616159, 0.08618535],
        [       nan,        nan,        nan, ..., 0.08609163,
         0.08609682, 0.08610656]]], dtype=float32)
Dimensions without coordinates: dim_0, dim_1, dim_2
Attributes:
    dimension_names:  ['time', 'mesh2d_nFaces', 'mesh2d_nLayers']
    variable_name:    mesh2d_sa1

In [28]:
# Select the desired slice using `isel`
# Adjust the variable and dimension names if they are slightly different in your data
sel_data_xr = data_xr['mesh2d_sa1'].isel(time=5, mesh2d_nLayers=45)

KeyError: 'mesh2d_sa1'

In [ ]:
# Ensure you have the correct variable for plotting
# Assuming mesh2d_sa1 is stored with a UGRID format and ugrid plot is available
data_xr.ugrid.plot()

print("Plot generated successfully.")